## CONFIG

In [ ]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup

from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [10]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
menu_esempio = "../Hackapizza Dataset/Menu/Anima Cosmica.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [4]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

## METODI

In [5]:
def estrai_relazioni(testo, categorie, llm):
    
    # Prompt migliorato per il modello AI
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti e Tecniche. 
    
    Mi serve che analizzi il testo e identifichi le RELAZIONI che legano le seguenti CATEGORIE all'interno del testo.
    
    Esempi di relazioni sono: DISTANTE, CONTIENE, PREPARATO 

    CATEGORIE: {categorie}

    Una relazione può essere tra una coppia di categorie o tra una categoria e se stessa.

    Voglio che mi ritorni un JSON con:
    - La chiave che indica il nome della relazione, che deve essere una SINGOLA PAROLA
    - La lista delle cateogire coinvolte nella relazione: una relazione può essere tra una coppia di categorie o tra una categoria e se stessa.

    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.

    Testo:
    {testo}
    """

    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        sostanze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle sostanze:", content)
        sostanze = {}

    return sostanze

In [6]:
def combina_punteggi(json_list, llm):
    """Combina i punteggi delle entità da più JSON, normalizzando sinonimi tramite LLM."""
    punteggi_totali = {}
    conteggio = {}
    
    for entita_manuale in json_list:
        for entita, punteggio in entita_manuale.items():
            if entita in punteggi_totali:
                punteggi_totali[entita] += punteggio
                conteggio[entita] += 1
            else:
                punteggi_totali[entita] = punteggio
                conteggio[entita] = 1
    
    punteggi_medi = {entita: punteggi_totali[entita] / conteggio[entita] for entita in punteggi_totali}
    
    # Normalizzazione con LLM
    prompt = f"""
    Data la seguente lista di categoria con punteggi, RAGGRUPPA SINONIMI e parole che hanno circa lo stesso significato sotto un'unica categoria (che deve essere una SINGOLA PAROLA).
    Restituisci il risultato come JSON con le categorie normalizzate e i punteggi medi aggregati.
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    
    {json.dumps(punteggi_medi, indent=2, ensure_ascii=False)}
    """
    
    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        sostanze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle sostanze:", content)
        sostanze = {}

    return sostanze
    #entita_normalizzate = json.loads(response.strip())
    #return dict(sorted(entita_normalizzate.items(), key=lambda item: item[1], reverse=True))

## MAIN

In [7]:
with open('entita_filtrate.txt', 'r', encoding='utf-8') as file:
    entita_filtrate = [line.strip() for line in file if line.strip()]

print(entita_filtrate)

['Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Piatto', 'Ristorante', 'Pianeta']


In [8]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo
entita_domande = estrai_relazioni(testo=testo_domande, categorie=entita_filtrate, llm=llm)
print("Entità CSV:", entita_domande)

Entità CSV: {'contiene': ['Piatto', 'Ingrediente'], 'preparato': ['Piatto', 'Tecnica'], 'servito': ['Piatto', 'Ristorante'], 'localizzato': ['Ristorante', 'Pianeta'], 'richiede': ['Tecnica', 'Licenza'], 'appartiene': ['Piatto', 'Ordine'], 'dista': ['Ristorante', 'Pianeta'], 'incluse': ['Ristorante', 'Piatto'], 'escluso': ['Piatto', 'Ingrediente'], 'utilizza': ['Tecnica', 'Ingrediente'], 'creato': ['Piatto', 'Ristorante'], 'necessita': ['Licenza', 'Tecnica'], 'serviti': ['Ristorante', 'Pianeta'], 'preparati': ['Piatto', 'Tecnica'], 'combina': ['Piatto', 'Ingrediente'], 'include': ['Piatto', 'Ingrediente'], 'evita': ['Piatto', 'Ingrediente'], 'fonte': ['Ingrediente', 'Pianeta'], 'associato': ['Ristorante', 'Pianeta']}


In [11]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo
entita_domande = estrai_relazioni(testo=testo_domande, categorie=entita_filtrate, llm=llm)
print("Entità CSV:", entita_domande)

Entità CSV: {'CONTIENE': ['Piatto', 'Ingrediente'], 'PREPARATO': ['Piatto', 'Tecnica'], 'OFFERTA': ['Ristorante', 'Piatto'], 'SITUATO': ['Ristorante', 'Pianeta'], 'RICHIEDE': ['Piatto', 'Licenza'], 'APPARTIENE': ['Ordine', 'Piatto']}


In [13]:
# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo
entita_pianeti = estrai_relazioni(testo=testo_pianeti, categorie=entita_filtrate, llm=llm)
print("Entità CSV:", entita_pianeti)

Entità CSV: {'DISTANTE': ['Pianeta', 'Pianeta']}


In [15]:
# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
entita_manuale = estrai_relazioni(testo=testo_manuale, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_manuale)

Entità PDF: {'utilizza': ['Tecnica', 'Ingrediente'], 'prepara': ['Tecnica', 'Piatto'], 'contiene': ['Piatto', 'Ingrediente'], 'situato': ['Ristorante', 'Pianeta'], 'serve': ['Ristorante', 'Piatto'], 'richiede': ['Tecnica', 'Licenza'], 'appartiene_piatto_ord': ['Piatto', 'Ordine'], 'appartiene_ristorante_ord': ['Ristorante', 'Ordine'], 'regola': ['Ordine', 'Licenza'], 'implementa': ['Ordine', 'Tecnica'], 'influenzato': ['Piatto', 'Pianeta']}


In [19]:
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)
entita_codice = estrai_relazioni(testo=testo_codice, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_codice)

Entità PDF: {'requires': ['Tecnica', 'Licenza'], 'belongs_to': ['Ordine', 'Pianeta'], 'prepares': ['Tecnica', 'Piatto'], 'located_on': ['Ristorante', 'Pianeta'], 'contains': ['Piatto', 'Ingrediente'], 'regulated_by': ['Ingrediente', 'Licenza'], 'implements': ['Ristorante', 'Tecnica'], 'follows': ['Ordine', 'Tecnica']}


In [23]:
# Esegui l'analisi del PDF
testo_menu = estrai_testo_pdf(menu_esempio)
entita_menu = estrai_relazioni(testo=testo_menu, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_menu)

Entità PDF: {'contiene': [['Ristorante', 'Piatto'], ['Piatto', 'Ingrediente']], 'preparato': [['Piatto', 'Tecnica']], 'situato': [['Ristorante', 'Pianeta']], 'possiede': [['Ristorante', 'Licenza']]}


In [ ]:
# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
entita_blog1 = estrai_relazioni(testo=testo_blog1, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_blog1)


Entità PDF: {'Ristorante': 35.0, 'Piatti': 25.0, 'Ingrediente': 20.0, 'Chef': 10.0, 'Tecnica': 10.0}


In [ ]:
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)
entita_blog2 = estrai_relazioni(testo=testo_blog2, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_blog2)

Entità PDF: {'Ristorante': 30.0, 'Ingrediente': 25.0, 'Piatto': 20.0, 'Tecnica': 10.0, 'Chef': 5.0, 'Pianeta': 5.0, 'Critico': 3.0, 'Voto': 2.0}


In [75]:
json_list = [entita_codice, entita_domande, entita_manuale, entita_menu, entita_pianeti, entita_blog1, entita_blog2]
punteggi_combinati = combina_punteggi(json_list, llm)
print("Punteggi combinati:", punteggi_combinati)

Punteggi combinati: {'Sostanza': 30.0, 'Tecnica': 26.25, 'Licenza': 22.75, 'Ordine': 10.0, 'Ingrediente': 28.125, 'Piatto': 40.0, 'Ristorante': 29.0, 'Pianeta': 40.0, 'Cottura': 15.0, 'Preparazione': 8.0, 'Galassia': 5.0, 'Menu': 12.0, 'Chef': 8.333333333333334, 'Critico': 3.0, 'Voto': 2.0}


In [ ]:
# Filtra le entità con punteggio superiore a 20 -> !TO ADAPT
entita_filtrate = [entita for entita, punteggio in punteggi_combinati.items() if punteggio >= 10]

print(entita_filtrate)

['Sostanza', 'Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Piatto', 'Ristorante', 'Pianeta', 'Cottura', 'Menu']


In [84]:
entita_filtrate.remove('Sostanza')
entita_filtrate.remove('Menu')
entita_filtrate.remove('Cottura')

ValueError: list.remove(x): x not in list

In [86]:
with open('entita_filtrate.txt', 'w') as file:
    for entita in entita_filtrate:
        file.write(f"{entita}\n")

## Creazione Collegamenti 

In [19]:
relazioni = {'CONTIENE': ['Piatto', 'Ingrediente'], 'PREPARATO': ['Piatto', 'Tecnica'], 'OFFERTA': ['Ristorante', 'Piatto'], 'SITUATO': ['Ristorante', 'Pianeta'], 'RICHIEDE': ['Piatto', 'Licenza']}#, 'APPARTIENE': ['Ordine', 'Piatto']}
relazioni

{'CONTIENE': ['Piatto', 'Ingrediente'],
 'PREPARATO': ['Piatto', 'Tecnica'],
 'OFFERTA': ['Ristorante', 'Piatto'],
 'SITUATO': ['Ristorante', 'Pianeta'],
 'RICHIEDE': ['Piatto', 'Licenza']}

In [20]:
class RelationshipExtractor:
    def __init__(self, llm, entities, relations_schema):
        self.llm = llm
        self.entities = entities  # Lista pre-validata dal KB
        self.relations_schema = relations_schema  # Es: {"CONTINE": ["Menu", "Piatto"], ...}

    def chunk_text(self, text: str, chunk_size=4000) -> List[Dict]:
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=500,
            separators=["\n\n", "\. ", "! ", "\? ", "\n", " "]
        )
        return [
            {
                "text": chunk,
                "entities": self._find_entities_in_chunk(chunk),
                "char_range": (i, i+len(chunk))
            }
            for i, chunk in enumerate(splitter.split_text(text))
        ]

    def _find_entities_in_chunk(self, chunk: str) -> List[str]:
        return [entity for entity in self.entities if re.search(rf'\b{re.escape(entity)}\b', chunk, flags=re.IGNORECASE)]


In [21]:
RELATION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """Sei un analista di documenti specializzato in ristorazione spaziale. 
     Identifica SOLO relazioni esplicitamente menzionate nel testo seguendo queste regole:
     
     Schema relazioni consentite:
     {relations_schema}

     Formato risposta:
     {{ 
       "relazioni": [
         {{ 
           "tipo": "nome_relazione",
           "sorgente": "entità_sorgente",
           "target": "entità_target",
           "evidenza": "testo rilevante",
           "confidence": 0-1
         }}
       ]
     }}"""),
    ("human", "Testo da analizzare:\n{chunk_text}\n\nEntità presenti nel testo: {chunk_entities}")
])


In [22]:
def validate_relationship(rel: Dict, current_graph: Graph) -> bool:
    # 1. Coerenza con lo schema
    if rel["tipo"] not in RELATIONS_SCHEMA:
        return False
    
    # 2. Esistenza entità nel grafo
    if not current_graph.nodes.match(rel["sorgente"]).exists():
        return False
    if not current_graph.nodes.match(rel["target"]).exists():
        return False
    
    # 3. Controllo duplicati
    existing = current_graph.relationships.match(
        nodes=[rel["sorgente"], rel["target"]], 
        r_type=rel["tipo"]
    )
    
    # 4. Soglia confidence
    return rel["confidence"] >= 0.7 and not existing


In [23]:
def upload_relationships(tx, rel_list):
    query = """
    UNWIND $rels AS rel
    MATCH (sorgente:Entity {name: rel.sorgente})
    MATCH (target:Entity {name: rel.target})
    MERGE (sorgente)-[r:REL_TYPE]->(target)
    SET r += { 
        evidence: rel.evidenza,
        sources: coalesce(r.sources, []) + rel.sources,
        confidence: (coalesce(r.confidence, 0) * coalesce(size(r.sources), 0) + rel.confidence) / (coalesce(size(r.sources), 0) + 1)
    }
    """
    tx.run(query, parameters={"rels": rel_list, "REL_TYPE": rel["tipo"]})


In [25]:
# Configurazioni (da adattare)
CONFIG = {
    "chunk_size": 1200,
    "min_confidence": 0.65,
    "relations_schema": {
        "CONTIENE": {"allowed_sources": ["Menu", "Pianeta"], "allowed_targets": ["Piatto", "Ingrediente"]},
        "DISTANTE": {"allowed_sources": ["Pianeta"], "allowed_targets": ["Pianeta"]},
        "PREPARATO_CON": {"allowed_sources": ["Piatto", "Chef"], "allowed_targets": ["Ingrediente", "Tecnica"]}
    }
}

def main_processing_flow(document_paths: list, neo4j_graph: Graph):
    # Inizializza estrattore
    extractor = RelationshipExtractor(
        llm=llm,
        entities=entita_filtrate,
        relations_schema=CONFIG["relations_schema"]
    )
    
    total_relationships = []
    
    for doc_path in document_paths:
        try:
            # Caricamento documento
            if doc_path.endswith('.pdf'):
                testo = estrai_testo_pdf(doc_path)
            elif doc_path.endswith('.html'):
                testo = estrai_testo_html(doc_path)
            elif doc_path.endswith('.csv'):
                df = pd.read_csv(doc_path)
                testo = df.to_string()
            else:
                continue
                
            # Chunking + entity linking
            chunks = extractor.chunk_text(testo)
            
            # Processa ogni chunk
            for chunk in chunks:
                try:
                    print()
                    # Estrazione relazioni
                    rel_prompt = RELATION_PROMPT.format(
                        relations_schema=json.dumps(CONFIG["relations_schema"], indent=2),
                        chunk_text=chunk["text"],
                        chunk_entities=", ".join(chunk["entities"])
                    )
                    print(f"{rel_prompt}")
                    
                    response = llm.invoke(rel_prompt)
                    print(f"{response}")
                    relationships = json.loads(response.content.strip())["relazioni"]
                    print(f"{relationships}")
           
                    # Aggiungi metadati contestuali
                    for rel in relationships:
                        rel.update({
                            "source_doc": os.path.basename(doc_path),
                            "char_start": chunk["char_range"][0],
                            "char_end": chunk["char_range"][1]
                        })
                    
                    # Filtraggio e validazione
                    valid_rels = [
                        rel for rel in relationships 
                        if (validate_relationship(rel, neo4j_graph) and 
                            rel["confidence"] >= CONFIG["min_confidence"])
                    ]
                    
                    total_relationships.extend(valid_rels)
                    
                except Exception as chunk_error:
                    print(f"Errore processing chunk: {str(chunk_error)}")
                    continue
                    
        except Exception as doc_error:
            print(f"Errore processing documento {doc_path}: {str(doc_error)}")
            continue
    
    # Upload batch a Neo4j (transazionale)
    if total_relationships:
        batch_size = 50
        for i in range(0, len(total_relationships), batch_size):
            batch = total_relationships[i:i+batch_size]
            
            try:
                with neo4j_graph.begin() as tx:
                    query = """
                    UNWIND $batch AS rel
                    MATCH (sorgente:Entity {name: rel.sorgente})
                    MATCH (target:Entity {name: rel.target})
                    MERGE (sorgente)-[r:REL_TYPE]->(target)
                    SET r += {
                        evidences: coalesce(r.evidences, []) + [rel.evidenza],
                        sources: coalesce(r.sources, []) + [rel.source_doc],
                        confidence: (coalesce(r.confidence, 0) * coalesce(size(r.evidences), 0) + rel.confidence) / 
                                   (coalesce(size(r.evidences), 0) + 1),
                        char_range: rel.char_start + '-' + rel.char_end
                    }
                    """
                    tx.run(query, {
                        "batch": batch,
                        "REL_TYPE": Relationship.type(rel["tipo"])
                    })
                    
            except Exception as neo_error:
                print(f"Neo4j batch error: {str(neo_error)}")
                # Logica di retry opzionale
                
    return len(total_relationships)

# Esecuzione
if __name__ == "__main__":
    documenti = [
       codice_galattico,
       manuale_cucina,
        menu_esempio,
    ]
    
    neo4j_conn = Graph(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    
    total_rels = main_processing_flow(documenti, neo4j_conn)
    print(f"Caricate {total_rels} relazioni nel grafo")



System: Sei un analista di documenti specializzato in ristorazione spaziale. 
     Identifica SOLO relazioni esplicitamente menzionate nel testo seguendo queste regole:
     
     Schema relazioni consentite:
     {
  "CONTIENE": {
    "allowed_sources": [
      "Menu",
      "Pianeta"
    ],
    "allowed_targets": [
      "Piatto",
      "Ingrediente"
    ]
  },
  "DISTANTE": {
    "allowed_sources": [
      "Pianeta"
    ],
    "allowed_targets": [
      "Pianeta"
    ]
  },
  "PREPARATO_CON": {
    "allowed_sources": [
      "Piatto",
      "Chef"
    ],
    "allowed_targets": [
      "Ingrediente",
      "Tecnica"
    ]
  }
}

     Formato risposta:
     { 
       "relazioni": [
         { 
           "tipo": "nome_relazione",
           "sorgente": "entità_sorgente",
           "target": "entità_target",
           "evidenza": "testo rilevante",
           "confidence": 0-1
         }
       ]
     }
Human: Testo da analizzare:
Gazzetta u(iciale del Consiglio Intergalattico   789

KeyboardInterrupt: 